In [ ]:
"""
Financial Analysis and Visualization Script

This script performs financial data analysis and visualization using Python libraries such as yfinance, Matplotlib, and mpl_finance. It loads historical stock data, identifies support and resistance levels, and plots candlestick charts with these levels.

The script consists of several functions:
1. `load_data(ticker, start_date, end_date)`: Loads historical stock data for a given ticker symbol within a specified date range. Drops unnecessary columns and returns a DataFrame.

2. `plot_candlestick_chart(df, save_filename=None)`: Plots a candlestick chart of the stock data. Optionally, saves the chart to a file.

3. `isSupport(df, i)`: Determines if a data point is a support level based on specific criteria.

4. `isResistance(df, i)`: Determines if a data point is a resistance level based on specific criteria.

5. `find_support_resistance_levels(df)`: Finds and stores support and resistance levels within the stock data.

6. `plot_support_resistance_chart(df, levels, levelr, save_filename=r'Images/sup_res.png')`: Plots a candlestick chart with support and resistance levels highlighted.

7. `filter_support_resistance_levels(df, support_levels, resistance_levels)`: Filters support and resistance levels based on average candle size.

8. `isFarFromLevel(l, avg_candlesize, updated_support_levels, updated_resistance_levels)`: Checks if a level is far from existing support or resistance levels.

9. `main()`: The main function of the script. Loads data, plots the initial candlestick chart, finds support and resistance levels, filters them, and plots the final chart.

Usage:
- Modify the `ticker_symbol`, `start_date`, and `end_date` variables in the `main()` function to analyze different stocks and date ranges.
- Execute the script to generate candlestick charts with support and resistance levels.

Note: Ensure that the necessary Python libraries (PyQt5, numpy, pandas, yfinance, mpl_finance, Matplotlib, IPython, talib) are installed before running the script.

"""

# The code implementation follows this docstring.


In [ ]:
import PyQt5
%matplotlib qt5

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
import mpl_finance as mpf
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from IPython.display import Image
import talib

In [ ]:
def load_data(ticker, start_date, end_date):
    tickerSymbol = ticker
    tickerData = yf.Ticker(tickerSymbol)
    
    # Retrieve historical data for the specified date range    
    df = tickerData.history(period="1d", start=start_date, end=end_date)
    
    # Convert date index to a numerical format for plotting
    df['Date'] = mdates.date2num(df.index)
    
     # Check if the DataFrame is empty, return it immediately if it is
    if df.empty:
        return df
    
    # Check if specific columns exist before dropping them
    columns_to_drop = ['Stock Splits', 'Dividends']
    existing_columns = df.columns.tolist()
    for column in columns_to_drop:
        if column in existing_columns:
            df.drop(column, axis=1, inplace=True)
    
    return df

In [ ]:
# Define a function to plot a candlestick chart
def plot_candlestick_chart(df, save_filename=None):
    # Set the figure size and font size for the plot
    plt.rcParams['figure.figsize'] = [12, 12]
    plt.rc('font', size=14)
    
    # Create a new figure and axis for the candlestick chart
    fig, ax = plt.subplots()
    
    # Generate a candlestick chart using the mpl_finance library
    mpf.candlestick_ohlc(ax, df[['Date', 'Open', 'High', 'Low', 'Close']].values,
                         width=0.8, colorup='green', colordown='red')
    
    # Format the date on the x-axis
    date_format = mdates.DateFormatter('%Y-%m-%d')
    ax.xaxis.set_major_formatter(date_format)
    
    # Rotate and format date labels for better readability
    fig.autofmt_xdate()
    
    # Set the title, x-axis label, and y-axis label for the chart
    ax.set_title('Candlestick Chart')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price ($)')
    
    # Save the chart to a file if a save_filename is provided, otherwise display it
    if save_filename:
        fig.savefig(save_filename)
    else:
        plt.show()


In [ ]:
# Define functions for support levels in the data
def isSupport(df, i):
    support = (
        df['Low'][i] < df['Low'][i - 1]
        and df['Low'][i] < df['Low'][i + 1]
        and df['Low'][i + 1] < df['Low'][i + 2]
        and df['Low'][i - 1] < df['Low'][i - 2]
    )
    return support

In [ ]:
# Define function to identify resistance levels in the data
def isResistance(df, i):
    resistance = (
        df['High'][i] > df['High'][i - 1]
        and df['High'][i] > df['High'][i + 1]
        and df['High'][i + 1] > df['High'][i + 2]
        and df['High'][i - 1] > df['High'][i - 2]
    )
    return resistance

In [ ]:
# Find and store support and resistance levels
def find_support_resistance_levels(df):
    resistance_levels = []  # Initialize an empty list to store resistance levels
    support_levels = []     # Initialize an empty list to store support levels

    # Loop through the DataFrame rows, excluding the first two and last two rows
    for i in range(2, df.shape[0] - 2):
        if isSupport(df, i):  # Check if the current row represents a support level
            support_levels.append((i, df['Low'][i]))  # Append the index and low price to support_levels list
        elif isResistance(df, i):  # Check if the current row represents a resistance level
            resistance_levels.append((i, df['High'][i]))  # Append the index and high price to resistance_levels list

    return support_levels, resistance_levels  # Return the lists of support and resistance levels


In [ ]:
# Define a function to plot a candlestick chart with support and resistance levels
def plot_support_resistance_chart(df, levels, levelr, save_filename=r'Images/sup_res.png'):
    # Create a new figure and axis for the plot
    fig, ax = plt.subplots()
    
    # Create a candlestick chart using the mpl_finance library
    mpf.candlestick_ohlc(ax, df[['Date', 'Open', 'High', 'Low', 'Close']].values, width=0.8, colorup='green', colordown='red', alpha=0.8)
    
    # Define the date format for the x-axis labels
    date_format = mdates.DateFormatter('%d %b %Y')
    ax.xaxis.set_major_formatter(date_format)
    
    # Automatically format the x-axis date labels for better readability
    fig.autofmt_xdate()
    
    # Set the title and labels for the chart
    ax.set_title('Support/Resistance Candlestick Chart', color='#800080', bbox=dict(facecolor='lightgrey', edgecolor='black'))
    ax.set_xlabel('Date', color='#800080')
    ax.set_ylabel('Price ($)', color='#800080')
    
    # Ensure tight layout for proper spacing
    fig.tight_layout()

    # Define labels for support and resistance lines
    support_label = 'Support'
    resistance_label = 'Resistance'

    # Create support lines on the chart based on detected support levels
    support_lines = []
    for level in levels:
        line = plt.hlines(level[1], xmin=df['Date'][level[0]], xmax=max(df['Date']), colors='blue', label=support_label)
        support_lines.append(line)

    # Create resistance lines on the chart based on detected resistance levels
    resistance_lines = []
    for level in levelr:
        line = plt.hlines(level[1], xmin=df['Date'][level[0]], xmax=max(df['Date']), colors='#DEC20B', label=resistance_label)
        resistance_lines.append(line)

    # Define legend lines for support and resistance
    legend_lines = [Line2D([0], [0], color='blue', lw=2, label=support_label), Line2D([0], [0], color='#DEC20B', lw=2, label=resistance_label)]
    
    # Add the legend to the chart with specified properties
    ax.legend(handles=legend_lines, facecolor='lightgrey', framealpha=1.0)
    
    # Save the chart as an image file if specified, or display it
    if save_filename:
        fig.savefig(save_filename)
    else:
        plt.show()


In [ ]:
# Define a function to filter support and resistance levels based on average candle size
def filter_support_resistance_levels(df, support_levels, resistance_levels):
    # Calculate the average candle size
    avg_candlesize = np.mean(df['High'] - df['Low'])
    
    # Initialize lists to store updated support and resistance levels
    updated_support_levels = []
    updated_resistance_levels = []

    # Iterate through the DataFrame to identify new support and resistance levels
    for i in range(2, df.shape[0] - 2):
        if isSupport(df, i):  # Check if the current point is a support level
            l = df['Low'][i]  # Get the low price at the current point
            # Check if the support level is far from existing levels
            if isFarFromLevel(l, avg_candlesize, updated_support_levels, updated_resistance_levels):
                updated_support_levels.append((i, l))  # Store the new support level
        
        elif isResistance(df, i):  # Check if the current point is a resistance level
            l = df['High'][i]  # Get the high price at the current point
            # Check if the resistance level is far from existing levels
            if isFarFromLevel(l, avg_candlesize, updated_support_levels, updated_resistance_levels):
                updated_resistance_levels.append((i, l))  # Store the new resistance level
            
    return updated_support_levels, updated_resistance_levels


In [ ]:
# Define a function to check if a level is far from existing support or resistance levels
def isFarFromLevel(l, avg_candlesize, updated_support_levels, updated_resistance_levels):
    
    # check if l is far from support levels
    far_from_support = np.sum([abs(l-x[1]) < avg_candlesize for x in updated_support_levels]) == 0
    # check if l is far from resistance levels
    far_from_resistance = np.sum([abs(l-x[1]) < avg_candlesize for x in updated_resistance_levels]) == 0
             
    return far_from_support or far_from_resistance

In [ ]:
# Main function
def main():
    ticker_symbol = 'AAPL'
    start_date = '2022-03-08'
    end_date = '2023-03-08'

    df = load_data(ticker_symbol, start_date, end_date)
    
    # Plot the initial candlestick chart
    plot_candlestick_chart(df, r'Images/candlestick_chart.png')
    
    # Find and plot support and resistance levels
    support_levels, resistance_levels = find_support_resistance_levels(df)
    plot_support_resistance_chart(df, support_levels, resistance_levels, save_filename=r'Images/noisy_sup_res.png')
    
    # Filter support and resistance levels based on candle size and plot again
    updated_support_levels, updated_resistance_levels = filter_support_resistance_levels(df, support_levels, resistance_levels)
    plot_support_resistance_chart(df, updated_support_levels, updated_resistance_levels, save_filename=r'Images/clean_sup_res.png')


In [ ]:
# Entry point of the script
if __name__ == "__main__":
    main()